In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic

#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupyter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group1-Marketing-Analytics"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

In [0]:
admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
try:
    topics =[confluentTopicName]
    fs = admin_client.delete_topics(topics, request_timeout=30)

    for topic, f in fs.items():
        try:
            f.result()  # The result itself is None
            print("Topic {} deleted".format(topic))
        except Exception as e:
            print("Failed to delete topic {}: {}".format(topic, e))
except Exception as e:
    print(e)

Out[18]: 'try:\n topics =[confluentTopicName]\n fs = admin_client.delete_topics(topics, request_timeout=30)\n\n for topic, f in fs.items():\n try:\n f.result() # The result itself is None\n print("Topic {} deleted".format(topic))\n except Exception as e:\n print("Failed to delete topic {}: {}".format(topic, e))\nexcept Exception as e:\n print(e)'

In [0]:
topic_list = []

topic_list.append(NewTopic(confluentTopicName, 1, 3))
admin_client.create_topics(topic_list)
futures = admin_client.create_topics(topic_list)


try:
    record_metadata = []
    for k, future in futures.items():
        # f = i.get(timeout=10)
        print(f"type(k): {type(k)}")
        print(f"type(v): {type(future)}")
        # print(future.result())

except KafkaError as error:
    # Decide what to do if produce request failed...
    print(traceback.format_exc())
    result = 'Fail'
finally:
    print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
finally

--------------------------------------------------------------------------- 
 KafkaException Traceback (most recent call last)
 <command-2451669969096127> in <module> 
 12 print ( f"type(k): {type(k)}" ) 
 13 print ( f"type(v): {type(future)}" ) 
 ---> 14 print ( future . result ( ) ) 
 15 
 16 except KafkaError as error : 

 /usr/lib/python3.8/concurrent/futures/_base.py in result (self, timeout) 
 442 raise CancelledError ( ) 
 443 elif self . _state == FINISHED : 
 --> 444 return self . __get_result ( ) 
 445 else : 
 446 raise TimeoutError ( ) 

 /usr/lib/python3.8/concurrent/futures/_base.py in __get_result (self) 
 387 if self . _exception : 
 388 try : 
 --> 389 raise self . _exception
 390 finally : 
 391 # Break a reference cycle with the exception in self._exception 

 KafkaException : KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'Group1-Marketing-Analytics' already exists."}